In [1]:
from os import listdir
from collections import Counter, defaultdict
from functools import reduce
import json
import math
import operator

# Rank
* Train Test Score
* filter no_filter 0.699
* no_filter filter 0.9227608874281019

In [2]:
KEYWORD_PATH = '/home/fun/Atos/Steeve_data/filter_Dice/can/'
TEST_PATH = '/home/fun/Atos/Steeve_data/no_filter_Dice/can/'
score_file = 'tfidf_score_filter.json'

limit = None

In [ ]:
ls '/home/fun/Atos/Steeve_data/no_filter_Dice/can/'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

if __name__ == "__main__":
    ########### Get all fields counter and total counter
    index_to_field, field_to_index, fields_pl = {}, {}, []
    for i, file in enumerate(listdir(KEYWORD_PATH)):
        doc = json.loads(open(KEYWORD_PATH + file, 'r', encoding='utf8').read())
        posts = sum(doc.values(), [])
        posts = posts[:limit] if limit else posts
        print("posts length: {}".format(len(posts)))
        
        index_to_field[i] = file
        field_to_index[file] = i
        pls = [p for post in posts for p in post['PL']]
        fields_pl.append(' '.join(pls))
    ############
    
    
    ############ Count tfidf
    vectorizer  = CountVectorizer()  # 該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    transformer = TfidfTransformer() # 該類會統計每個詞語的tf-idf權值  
    tfidf = transformer.fit_transform(vectorizer.fit_transform(fields_pl)) # 第一個fit_transform是計算tf-idf，第二個fit_transform是將文本轉為詞頻矩陣  
    
    word = vectorizer.get_feature_names() # 獲取詞袋模型中的所有詞語  
    weight = tfidf.toarray()              # 將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  
    
    tfidf_score = defaultdict(lambda: defaultdict())
    for i in range(len(weight)):          # 打印每類文本的tf-idf詞語權重，第一個for遍歷所有文本，第二個for便利某一類文本下的詞語權重  
        print("-------這裡輸出第",i,u"類文本的詞語tf-idf權重------")
        for j in range(len(word)):
            tfidf_score[index_to_field[i]][word[j]] = weight[i][j]
            print(word[j], weight[i][j])
            
    with open(score_file, 'w', encoding='utf8') as ws:
        ws.write(json.dumps(tfidf_score))

In [ ]:
def predict():
    correct, total = 0, 0
    unexist = Counter()
    for i, file in enumerate(listdir(TEST_PATH)):
        print(file, index_to_field[i])
        
        doc = json.loads(open(TEST_PATH + file, 'r', encoding='utf8').read())
        posts = [post['PL'] for post in sum(doc.values(), [])]

        for post in posts:
            fields_score = defaultdict(lambda: 0.0)
            for pl in post:
                pl = pl.lower()
                if pl not in word:
                    unexist[pl] += 1
                    continue
                    
                for field, score in tfidf_score.items():
                    fields_score[field] += score[pl]
            
            try:
                index = max(fields_score.items(), key=operator.itemgetter(1))[0]
            except:
                print(fields_score)

            if index == index_to_field[i]: 
                correct += 1
            total += 1
            
    print(correct)
    print(total)
    print(correct / total)
    print(unexist)
    
predict()